In [3]:
import requests
import json
import datetime

vilage_weather_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?" #공공데이터 포탈을 통해 데이터 수집

service_key = "6ClTIhHLgyLqzpIO1EsI54T%2BD02t9STimsteXUUqMaqDJ17G5ylnSlE%2BJ7AsCtqWCyV2H9aOtmDpVGgRwzatoA%3D%3D"#API 인증키

today = datetime.datetime.today()
base_date = today.strftime("%Y%m%d") # 오늘 날짜 받아오기
base_time = "0500" # 날씨 값 설정

nx = "68" #위치 청주시 서원구 사창동
ny = "107"

payload = "serviceKey=" + service_key + "&" +\
    "dataType=json" + "&" +\
    "base_date=" + base_date + "&" +\
    "base_time=" + base_time + "&" +\
    "nx=" + nx + "&" +\
    "ny=" + ny

# 값 요청
res = requests.get(vilage_weather_url + payload)

items = res.json().get('response').get('body').get('items').get('item')

In [4]:
data = dict()
data['date'] = base_date

weather_data = dict()
for item in items:
    # 기온
    if item['category'] == 'T3H':
        weather_data['tmp'] = item['fcstValue']
    
    # 기상상태
    if item['category'] == 'PTY':
        
        weather_code = item['fcstValue']
        
        if weather_code == '1':
            weather_state = '비'
        elif weather_code == '2':
            weather_state = '비/눈'
        elif weather_code == '3':
            weather_state = '눈'
        elif weather_code == '4':
            weather_state = '소나기'
        else:
            weather_state = '없음'
        
        weather_data['code'] = weather_code
        weather_data['state'] = weather_state

data['weather'] = weather_data
data['weather']


{'code': '0', 'state': '없음', 'tmp': '24'}

In [45]:
rain_foods = "부대찌개,아구찜,해물탕,칼국수,수제비,짬뽕,우동,치킨,국밥,김치부침개,두부김치,파전".split(',')

In [46]:
# 네이버 인증
# https://developers.naver.com/apps
# 해당 사이트에서 로그인 후 "Cliend ID"와 "Client Secret"을 얻어오세요
ncreds = {
    "client_id": "h2hShXcfQlggujGmM3J8",      
    "client_secret" : "s3DPfBHzd1"
}
nheaders = {
    "X-Naver-Client-Id" : ncreds.get('client_id'),
    "X-Naver-Client-Secret" : ncreds.get('client_secret')
}

# 경우 1 : 비/눈/소나기           => rain_foods에 있는 맛집
# 경우 2 : 정상                   => 블로그 리뷰 순 맛집 추천

# weather_state
if data.get('weather').get('code') != '0':
    weather_state = '1'
else:
    weather_state = '2'

In [47]:
import random
# random.sample(x, k=len(x)) 무작위로 리스트 섞기

foods_list = None

# 경우 1, 2
if weather_state == '1':
    foods_list = random.sample(rain_foods, k=len(rain_foods))
else:
    foods_list = ['']

foods_list
# ['쌀국수', '굴', '콩나물국밥', '마라탕', '고등어']


food_list = ['쌀국수', '굴', '콩나물국밥', '마라탕', '고등어']



In [48]:
import urllib

# 네이버 지역 검색 주소
naver_local_url = "https://openapi.naver.com/v1/search/local.json?"

# 검색에 사용될 파라미터
# 정렬 sort : 리뷰순(comment)
# 검색어 query : 인코딩된 문자열
params_format = "sort=comment&query="

# 위치는 사용자가 사용할 지역으로 변경가능
location = "충북대"

# 추천된 맛집을 담을 리스트
recommands = []
for food in foods_list:
    # 검색어 지정
    query = location + " " + food + " 맛집"
    enc_query = urllib.parse.quote(query)
    params = params_format + enc_query
    
    # 검색
    # headers : 네이버 인증 정보
    res = requests.get(naver_local_url + params, headers=nheaders)
    
    # 맛집 검색 결과
    result_list = res.json().get('items')

    # 경우 3 처리
    # 맛집 검색 결과에서 가장 상위 3개를 가져옴
    if weather_state == '3':
        for i in range(0,3):
            recommands.append(result_list[i])
        break
    
    # 경우 1,2 처리
    # 해당 음식 검색 결과에서 가장 상위를 가져옴
    if result_list:
        recommands.append(result_list[0])
        # 3개를 찾았다면 검색 중단
        if len(recommands) >= 3:
            break

In [49]:
app_key = "e611d4d685428e8ec80f37429a715ee4"#카카오 rest api 키
code = "Iguic0jXra4_ygKcJwv7eX1dZ-BSKIYhM1Puyl3zS-vF3DxDSMlz4bWsc_3hFicg-LmCKgo9dRkAAAFywW9gAg"
#카카오톡으로 자신에게 정보 전달 하도록 하는 연결키

In [50]:
url = "https://kauth.kakao.com/oauth/token"#카카오톡 토큰 url

data2 = {
    "grant_type" : "authorization_code",
    "client_id" : app_key, 
    "redirect_uri" : "https://localhost.com",
    "code"         : code
    
}
response = requests.post(url, data=data2)

tokens = response.json()

print(tokens)

{'access_token': 'GXgvmBsaqj45yd14IkaLtX-65dNEU93eH4pZUgopb1UAAAFywXBWAQ', 'token_type': 'bearer', 'refresh_token': '-k-pFvBV8n1rL-cPSBzTexy2I8wFfN5PXlCTfQopb1UAAAFywXBWAA', 'expires_in': 21599, 'scope': 'account_email talk_message profile', 'refresh_token_expires_in': 5183999}


In [51]:
#url = "https://kauth.kakao.com/oauth/token"
#data3 = {
#    "grant_type" : "refresh_token",
#    "client_id"  : app_key,
#    "refresh_token" : "LR2JqA6PXecQJK2VBnU8sC9evim2W5jnoK3dSwo9dNsAAAFyhJLrig"
#}
#response = requests.post(url, data=data3)
#tokens = response.json()
#print(tokens)

In [52]:
with open("kakao_token.json", "w") as fp:
    json.dump(tokens, fp)

In [53]:
#with open("kakao_token.json", "r") as fp:
 #   tokens = json.load(fp)

In [54]:
tokens

{'access_token': 'GXgvmBsaqj45yd14IkaLtX-65dNEU93eH4pZUgopb1UAAAFywXBWAQ',
 'token_type': 'bearer',
 'refresh_token': '-k-pFvBV8n1rL-cPSBzTexy2I8wFfN5PXlCTfQopb1UAAAFywXBWAA',
 'expires_in': 21599,
 'scope': 'account_email talk_message profile',
 'refresh_token_expires_in': 5183999}

In [55]:
kcreds = {
    "access_token" : tokens['access_token']
}

kheaders = {
    "Authorization" : "Bearer " + kcreds.get('access_token')
}

In [56]:
import json
import requests

# 카카오톡 URL 주소
kakaotalk_template_url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# 날씨 상세 정보 URL
weather_url = "https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query=%EB%82%A0%EC%94%A8"

# 날씨 정보 만들기 
text = f"""#날씨 정보 ({data['date']}) 
기온 : {data['weather']['tmp']} 
기우  : {data['weather']['state']} 
"""

# 텍스트 템플릿 형식 만들기
template = {
  "object_type": "text",
  "text": text,
  "link": {
    "web_url": weather_url,
    "mobile_web_url": weather_url
  },
  "button_title": "날씨 상세보기"
}

# JSON 형식 -> 문자열 변환
payload = {
    "template_object" : json.dumps(template)
}

# 카카오톡 보내기
res = requests.request("POST", kakaotalk_template_url, data =payload, headers=kheaders)

if res.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(res.json()))


메시지를 성공적으로 보냈습니다.


In [1]:
contents = []
template = {
    "object_type" : "list",
    "header_title" : "현재 날씨에 따른 음식 추천",
    "header_link" : {
        "web_url": weather_url,
        "mobile_web_url" : weather_url
    },
    "contents" : contents,
    "buttons" : [
        {
            "title" : "날씨 정보 상세보기",
            "link" : {
                "web_url": weather_url,
                "mobile_web_url" : weather_url
            }
        }
    ],
}


# contents 만들기
for place in recommands:
    title = place.get('title')  # 장소 이름
    # title : 태극쿵푸<b>마라탕</b>
    # html 태그 제거
    title = title.replace('<b>','').replace('</b>','')
    
    category = place.get('category')  # 장소 카테고리
    telephone = place.get('telephone')  # 장소 전화번호
    address = place.get('address')  # 장소 지번 주소

    # 각 장소를 클릭할 때 네이버 검색으로 연결해주기 위해 작성된 코드
    enc_address = urllib.parse.quote(address + ' ' + title)
    query = "query=" + enc_address

    # 음식 카테고리에 따라 해당 대표음식 이미지 출력(이미지 url이용)
    if '카페' in category:
        image_url = "https://freesvg.org/img/pitr_Coffee_cup_icon.png"
    elif '치킨' in category:
        image_url = "http://www.momstouch.co.kr/data/shopimages/xboard/menu/20170622893503.jpg"
    elif '전' in category:
        image_url = "http://www.sk5.co.kr/img_src/s600/a918/a9180025.jpg"
    elif '부침' in category:
        image_url = "http://www.sk5.co.kr/img_src/s600/a918/a9180025.jpg"
    elif '칼국수' in category:
        image_url = "http://www.sk5.co.kr/img_src/s600/1209/12091443.jpg"
    elif '찜' in category:
        image_url = "https://t1.daumcdn.net/cfile/tistory/26590D3C577B13FB30"
    elif '부대찌개' in category:
        image_url = "https://t1.daumcdn.net/cfile/tistory/235B103C577B13FB2F"
    elif '일식' in category:
        image_url = "http://www.simbata.co.kr/img_src/s600/1227/12273259.jpg"
    elif '고기' in category:
        image_url = "https://t1.daumcdn.net/liveboard/dailylife/8f41a57e8e0f4ce7878ffad0eda9e5bc.JPG"
    else:
        image_url = "https://pgnqdrjultom1827145.cdn.ntruss.com/img/0e/fe/0efeb7320298d35eb99b02e2af0cc0a6cdb87174cfd2f85e36f501a87efd6448_v1.jpg"

    # 전화번호가 있다면 제목과 함께 넣어줍니다.
    if telephone:
        title = title + "\ntel) " + telephone

    # 카카오톡 리스트 템플릿 형식에 맞춰줍니다.
    content = {
        "title": "[" + category + "] " + title,
        "description": ' '.join(address.split()[1:]),
        "image_url": image_url,
        "image_width": 50, "image_height": 50,
        "link": {
            "web_url": "https://search.naver.com/search.naver?" + query,
            "mobile_web_url": "https://search.naver.com/search.naver?" + query
        }
    }
    
    contents.append(content)

# JSON 형식 -> 문자열 변환
payload = {
    "template_object" : json.dumps(template)
}

# 카카오톡 보내기
res = requests.post(kakaotalk_template_url, data=payload, headers=kheaders)

if res.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(res.json()))


# # 맛집 위치를 카카오맵을 통해 카카오톡으로 전송

NameError: name 'weather_url' is not defined